In [1]:
import sys, pandas as pd
import torch

print("Python executable:", sys.executable)
print("Pandas version:", pd.__version__)
print("Torch version:", torch.__version__)


Python executable: C:\ProgramData\anaconda3\envs\derm7pt\python.exe
Pandas version: 2.3.2
Torch version: 2.8.0+cpu
